# ConncectBuildingCSV
This program parses CityGML and find gml_id of buildings at [lon, lat]s given by a CSV file.

Tested with PLATEAU CityGML for Kyoto city.

The parsing algorithm for CityGML is retrieved from plateauutils.

In [78]:
# @title Prepare Google Drive
from google.colab import drive
import warnings

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
path_home = "/content/drive/MyDrive/PLATEAU-AJARI/"

In [80]:
!pip install geojson

In [81]:
import numpy as np
import math
import geojson
from shapely import from_geojson
from progressbar import progressbar
from shapely.geometry import Polygon, Point
import xml.etree.ElementTree as ET
import zipfile
import csv

In [82]:
url_CityGML = "https://assets.cms.plateau.reearth.io/assets/38/a9bf00-0d6b-4b6c-9e2f-c8a72404da66/26100_kyoto-shi_2022_citygml_1_op.zip"
path_cgml = path_home
#parser = CityGMLParser(boundary)
#buildings = parser.download_and_parse(url_CityGML, path_cgml)

In [83]:
target_path = path_home+"26100_kyoto-shi_2022_citygml_1_op.zip"
hit_targets = []
bldg_list = {}
# namespaceを作成
ns = {
    "core": "http://www.opengis.net/citygml/2.0",
    "bldg": "http://www.opengis.net/citygml/building/2.0",
    "gml": "http://www.opengis.net/gml",
    "uro": "https://www.geospatial.jp/iur/uro/2.0"
}
with zipfile.ZipFile(target_path) as zf:
    for name in progressbar(zf.namelist()):
      #for target in self.targets:
      #    path = os.path.join("udx", "bldg", target + "_bldg")
      #    if name.find(path) >= 0 and
      if name.find("/udx/bldg/")>=0 and name.endswith(".gml"):
        hit_targets.append(name)
    for hit_target in progressbar(hit_targets):
      #bldg_list.append(ET.fromstring(zf.read(hit_target)))
      root = ET.fromstring(zf.read(hit_target))
      city_object_members = root.findall(".//core:cityObjectMember", ns)
      for city_object_member in city_object_members:
          # bldg:Buildingを取得
          building = city_object_member.find(".//bldg:Building", ns)
          # gml:idを取得
          gid = building.get("{http://www.opengis.net/gml}id")
          lod1_solid = city_object_member.find(".//bldg:lod1Solid", ns)
          pos_lists = lod1_solid.findall(".//gml:posList", ns)
          poi_list = pos_lists[0]
          numbers = [float(x) for x in poi_list.text.split()]
          array = np.array(numbers).reshape((-1, 3))
          new_array = np.delete(array, 2, axis=1)
          # Polygonを作成する際にlon, latの順番にする
          list_of_points = [sublist[::-1] for sublist in new_array.tolist()]
          polygon = Polygon(list_of_points)
          #tmp = building # building.find(".//bldg:lod0RoofEdg", ns)
          bldg_list[gid] = {"gid":gid, "polygon":polygon}

100% (104839 of 104839) |################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (238 of 238) |######################| Elapsed Time: 0:06:38 Time:  0:06:38


In [84]:
print(hit_targets)
print(len(hit_targets))
print(list(bldg_list.keys())[:10])
print(len(bldg_list))

['26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352558_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352567_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352568_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352576_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352577_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352586_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352587_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352588_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352597_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352598_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352599_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352691_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52352692_bldg_6697_op.gml', '26100_kyoto-shi_2022_citygml_1_op/udx/bldg/52353507_bldg_6697_

In [85]:
# from plateau-2023-uc01-satellite-analysis/PLATEAU-FloodSAR/plateau_floodsar_lib.py
__LATLIMIT = 85.0511
__YZERO = math.atanh(math.sin(math.radians(__LATLIMIT)))
#__TD = 256
def calc_xyz_from_lonlat(lon: float, lat: float, z: int):
    """緯度と経度を含むタイルのXYZ座標を計算する関数
    国土地理院用
    """
    #print(lon, lat, z)
    n = 2.0 ** z
    x = int((lon + 180.0) / 360.0 * n)
    y = int( (__YZERO-math.atanh(math.sin(math.radians(lat))))
        /(2.0*__YZERO) * n )
    return (x, y) #, (lon, lat)

In [86]:
print(list(bldg_list.values())[0]["polygon"].bounds)

(135.7276906568153, 34.88298188729769, 135.72779953554104, 34.88305584203023)


In [87]:
searchlist = {}
zoom = 16
for bb in progressbar(bldg_list.values()):
  xl, yl, xh, yh = bb["polygon"].bounds
  idxl, idyl = calc_xyz_from_lonlat(xl,yh,zoom)
  idxh, idyh = calc_xyz_from_lonlat(xh,yl,zoom)
  # assume the building crosses no more than one border of xyz tile
  # in lon or lat direction at given zoom level
  for xx in set([idxl, idxh]):
    if xx not in searchlist:
      searchlist[xx] = {}
    for yy in set([idyl, idyh]):
      if yy not in searchlist[xx]:
        searchlist[xx][yy] = []
      searchlist[xx][yy].append(bb["gid"])

100% (506471 of 506471) |################| Elapsed Time: 0:00:16 Time:  0:00:16


In [88]:
print(searchlist.keys())
print(searchlist[57476].keys())

dict_keys([57476, 57475, 57474, 57472, 57473, 57477, 57478, 57484, 57485, 57487, 57486, 57480, 57479, 57469, 57468, 57470, 57465, 57466, 57467, 57471, 57464, 57463, 57462, 57481, 57482, 57483, 57488, 57492, 57491, 57489, 57490, 57493, 57494, 57495, 57496])
dict_keys([25984, 25983, 25979, 25980, 25978, 25977, 25975, 25976, 25974, 25973, 25972, 25971, 25970, 25969, 25968, 25967, 25966, 25965, 25964, 25962, 25963, 25961, 25960, 25959, 25958, 25956, 25957, 25955, 25954, 25953, 25952, 25951, 25950, 25949, 25946, 25947])


In [89]:
filename = path_home + "KyotoBunkazai.csv"
with open(filename, encoding='utf8', newline='') as ifile:
    csvreader = list(csv.reader(ifile))
    header = csvreader[0]
    csvreader = csvreader[1:]
    #for row in csvreader:
    #    print(row)

In [90]:
#print(header)
print(csvreader)

[['102', '1985', '愛宕神社本殿', '', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／神社', '', '鎌倉後期', '19770128', '', '京都府', '京都府亀岡市千歳町国分', '', '愛宕神社', '', '35.04093223000000', '135.58701004000000'], ['102', '1988', '荒見神社本殿', '', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／神社', '', '桃山', '19060414', '', '京都府', '京都府城陽市富野荒見田', '', '荒見神社', '', '34.84096835000000', '135.78407307000000'], ['102', '1808', '安養寺宝塔', '', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／その他', '', '鎌倉後期', '19600209', '', '京都府', '京都府京都市東山区八坂鳥居前東入円山町', '', '安養寺', '', '35.00375699000000', '135.78426013000000'], ['102', '1927', '安楽寿院五輪塔', '', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／その他', '', '鎌倉後期', '19550202', '', '京都府', '京都府京都市伏見区竹田内畑町', '', '安楽寿院', '', '34.95329328000000', '135.75383267000000'], ['102', '1891', '為因寺宝篋印塔', '', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／その他', '', '鎌倉前期', '19550202', '', '京都府', '京都府京都市右京区梅ヶ畑奥殿町', '', '為因寺', '', '35.04725433000000', '135.68134072000000'], ['102', '2000', '伊佐家住宅（京都府八幡市上津屋）', '長蔵', '国宝・重要文化財（建造物）', '重要文化財', '近世以前／民家', '', '江戸末期', '1980121

In [91]:
for ii, ll in enumerate(header):
  print(f"{ii}: {ll}")

0: ﻿"台帳ID"
1: 管理対象ID
2: 名称
3: 棟名
4: 文化財種類
5: 種別1
6: 種別2
7: 国
8: 時代
9: 重文指定年月日
10: 国宝指定年月日
11: 都道府県
12: 所在地
13: 保管施設の名称
14: 所有者名
15: 管理団体又は責任者
16: 緯度
17: 経度


In [92]:
targets = [
    {"name":vv[2] if len(vv[3])==0 else vv[2]+"/"+vv[3], "era":vv[8], "lon":float(vv[17]), "lat":float(vv[16])} for vv in csvreader
]

In [93]:
print(targets)

[{'name': '愛宕神社本殿', 'era': '鎌倉後期', 'lon': 135.58701004, 'lat': 35.04093223}, {'name': '荒見神社本殿', 'era': '桃山', 'lon': 135.78407307, 'lat': 34.84096835}, {'name': '安養寺宝塔', 'era': '鎌倉後期', 'lon': 135.78426013, 'lat': 35.00375699}, {'name': '安楽寿院五輪塔', 'era': '鎌倉後期', 'lon': 135.75383267, 'lat': 34.95329328}, {'name': '為因寺宝篋印塔', 'era': '鎌倉前期', 'lon': 135.68134072, 'lat': 35.04725433}, {'name': '伊佐家住宅（京都府八幡市上津屋）/長蔵', 'era': '江戸末期', 'lon': 135.74655666, 'lat': 34.86473691}, {'name': '伊佐家住宅（京都府八幡市上津屋）/主屋', 'era': '江戸中期', 'lon': 135.74664385, 'lat': 34.8648971}, {'name': '伊佐家住宅（京都府八幡市上津屋）/内蔵', 'era': '江戸中期', 'lon': 135.74664791, 'lat': 34.86499038}, {'name': '伊佐家住宅（京都府八幡市上津屋）/乾蔵', 'era': '江戸末期', 'lon': 135.74646136, 'lat': 34.86508568}, {'name': '伊佐家住宅（京都府八幡市上津屋）/東蔵', 'era': '江戸中期', 'lon': 135.74664183, 'lat': 34.86509582}, {'name': '石田家住宅（京都府北桑田郡美山町）', 'era': '江戸前期', 'lon': 135.47523032, 'lat': 35.27502525}, {'name': '石田神社境内社恵比須神社本殿', 'era': '鎌倉後期', 'lon': 135.31131258, 'lat': 35.34027524}, {'nam

In [97]:
cntNF = 0
for tt in progressbar(targets):
  xx, yy = calc_xyz_from_lonlat(tt["lon"],tt["lat"],zoom)
  #print(xx, yy)
  pt = Point(tt["lon"], tt["lat"])
  if xx in searchlist and yy in searchlist[xx]:
    for bid in searchlist[xx][yy]:
      if pt.within(bldg_list[bid]["polygon"]):
        tt["gid"] = bid
        break
  if "gid" not in tt:
    tt["gid"] = "NotFound"
    cntNF += 1

100% (701 of 701) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


In [101]:
print(targets)
print(f"{cntNF}/{len(targets)}")

[{'name': '愛宕神社本殿', 'era': '鎌倉後期', 'lon': 135.58701004, 'lat': 35.04093223, 'gid': 'NotFound'}, {'name': '荒見神社本殿', 'era': '桃山', 'lon': 135.78407307, 'lat': 34.84096835, 'gid': 'NotFound'}, {'name': '安養寺宝塔', 'era': '鎌倉後期', 'lon': 135.78426013, 'lat': 35.00375699, 'gid': 'NotFound'}, {'name': '安楽寿院五輪塔', 'era': '鎌倉後期', 'lon': 135.75383267, 'lat': 34.95329328, 'gid': 'NotFound'}, {'name': '為因寺宝篋印塔', 'era': '鎌倉前期', 'lon': 135.68134072, 'lat': 35.04725433, 'gid': 'NotFound'}, {'name': '伊佐家住宅（京都府八幡市上津屋）/長蔵', 'era': '江戸末期', 'lon': 135.74655666, 'lat': 34.86473691, 'gid': 'NotFound'}, {'name': '伊佐家住宅（京都府八幡市上津屋）/主屋', 'era': '江戸中期', 'lon': 135.74664385, 'lat': 34.8648971, 'gid': 'NotFound'}, {'name': '伊佐家住宅（京都府八幡市上津屋）/内蔵', 'era': '江戸中期', 'lon': 135.74664791, 'lat': 34.86499038, 'gid': 'NotFound'}, {'name': '伊佐家住宅（京都府八幡市上津屋）/乾蔵', 'era': '江戸末期', 'lon': 135.74646136, 'lat': 34.86508568, 'gid': 'NotFound'}, {'name': '伊佐家住宅（京都府八幡市上津屋）/東蔵', 'era': '江戸中期', 'lon': 135.74664183, 'lat': 34.86509582, 'gid':

In [102]:
eralist = set([tt["era"] for tt in targets])

In [103]:
print(eralist)

{'鎌倉前期', '室町後期', '', '昭和', '江戸前期', '室町中期', '室町前期', '江戸中期', '江戸後期', '明治', '平安中期', '鎌倉後期', '桃山', '大正', '平安後期', '江戸末期', '平安前期'}


In [104]:
era_selected = ['鎌倉前期', '室町後期', '江戸前期', '室町中期', '室町前期', '江戸中期', '江戸後期', '平安中期', '鎌倉後期', '桃山', '平安後期', '江戸末期', '平安前期']

In [105]:
selected = [tt for tt in targets if tt["gid"]!="NotFound" and tt["era"] in era_selected]

In [107]:
print(selected)
print(len(selected))

[{'name': '裏千家住宅（京都府京都市上京区小川通寺之内上る本法寺前町）', 'era': '江戸末期', 'lon': 135.75365574, 'lat': 35.03462606, 'gid': 'bldg_24ad8263-adcc-43f9-aeaa-9e03fc8ac302'}, {'name': '雲竜院本堂', 'era': '江戸前期', 'lon': 135.78009822, 'lat': 34.97717099, 'gid': 'bldg_52c05c1b-52c3-434e-9027-812a8166f3a1'}, {'name': '燕庵', 'era': '江戸末期', 'lon': 135.75487748, 'lat': 34.99096433, 'gid': 'bldg_03cdd2d0-7965-4782-81eb-f03772397607'}, {'name': '黄梅院/本堂', 'era': '桃山', 'lon': 135.74619387, 'lat': 35.04174144, 'gid': 'bldg_ccb74373-c69e-448f-9e19-c9eb2fdc8dbb'}, {'name': '黄梅院/庫裏', 'era': '桃山', 'lon': 135.74609095, 'lat': 35.0417635, 'gid': 'bldg_ccb74373-c69e-448f-9e19-c9eb2fdc8dbb'}, {'name': '小川家住宅（京都府京都市中京区大宮通御池下る）/主屋', 'era': '江戸後期', 'lon': 135.74870031, 'lat': 35.01028042, 'gid': 'bldg_5f5ff309-dc95-4042-96bb-f2faca37c9e4'}, {'name': '小川家住宅（京都府京都市中京区大宮通御池下る）/西土蔵', 'era': '江戸後期', 'lon': 135.74838001, 'lat': 35.01029721, 'gid': 'bldg_b97a61f1-eb04-459d-b577-19ce678d4291'}, {'name': '小川家住宅（京都府京都市中京区大宮通御池下る）/北土蔵', 'era': '江

In [112]:
# Generate Style.json
path_style_json = path_home+"style_edo.json"
with open(path_style_json,"w") as ofile:
  ofile.write("""{"show":"(true)","color":{"conditions":[""")
  for tt in selected:
    ofile.write("""["${feature['gml_id']}==='""")
    ofile.write(tt["gid"])
    ofile.write("""'","color('#FF00FF70')"],""")
  ofile.write("""["true","color('#00000000')"]]}}""")